In [1]:
from sklearn import svm
import random
import numpy as np
import matplotlib.pyplot as plt
from smt.surrogate_models import KRG
from cec2017.functions import all_functions

In [2]:
def create_pop(size, n, min_val, max_val):
    population = np.zeros((size, n))
    for i in range(size):
        population[i] = np.random.uniform(min_val, max_val, size=n)
    return population


def mutation(sort_index_fit, population, F, pop_size, j):
    a = j
    b = j
    c = j
    while a == j:
        a = sort_index_fit[np.random.randint(0, pop_size*0.2)]
    while b == j or b == a:
        b = np.random.randint(0, pop_size)
    while c == j or c == b or c == a:
        c = np.random.randint(0, pop_size)
    #print("mut", "a:", population[a], "b:", population[b], "c:", population[c], "j:", population[j])
    #print(population)
    mutated_vector = population[j] + F * (population[a] - population[j]) + F * (population[b] - population[c])
    return mutated_vector


def crossover(target, mutant, crossover_prob):
    crossed = np.zeros(n)
    indx = np.random.randint(0, n)
    for i in range(n):
        if random.random() < crossover_prob or i == indx:
            crossed[i] = mutant[i]
        else:
            crossed[i] = target[i]
    return crossed


def func(x):
    return x[0] ** 2 + x[1] ** 2

def rosenbrock(x):
    x1, x2 = x[0], x[1]
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2

In [30]:
def differential_evolution(mode, objective_func, pop_size, n_params, min_val, max_val, num_calculate, num_calc_surr):
    calc = pop_size
    iter = 0
    population = create_pop(pop_size, n_params, min_val, max_val)
    fit_pop = objective_func(population)
    count_calculate = pop_size
    sort_index_fit = np.argsort(fit_pop.flatten())
    H = 5
    MF = np.ones(H) * 0.5
    MCr = np.ones(H) * 0.5
    Sf = np.zeros(pop_size)
    Scr = np.zeros(pop_size)
    d_f = np.zeros(pop_size)
    
    surr = 0
    xt = np.copy(population)
    yt = np.copy(fit_pop).reshape(pop_size, 1)
    sm = KRG(theta0=[1e-2], print_global = False)
    sm.set_training_values(xt, yt)
    sm.train()

    while calc < num_calculate:
        population_surr = create_pop(pop_size, n_params, min_val, max_val)
        fitness_surr = np.zeros(pop_size)
        
        while surr < num_calc_surr:
            count = 0
            iter += 1
            for i in range(pop_size):
                fit_model = sm.predict_values
                fitness_surr[i] = fit_model(population_surr[i].reshape(1, -1))
                surr += 1

            for idx, ind in enumerate(population):
                target = ind
                r = np.random.randint(0, H)
                F = -1
                while F < 0 or F > 1:
                    F = np.random.standard_cauchy() * 0.1 + MF[r]

                crossover_prob = -1
                while crossover_prob < 0 or crossover_prob > 1:
                    crossover_prob = np.random.normal(MCr[r], 0.1)

                while True:
                    index = random.randint(0, pop_size - 1)
                    if index != idx:
                        break

                mutant = mutation(sort_index_fit, population, F, pop_size, idx)
                crossed = crossover(target, mutant, crossover_prob)
                corrected_vector = np.array([])
                for i in range(len(crossed)):
                    if crossed[i] < min_val:
                        corrected_vector = np.append(corrected_vector, (target[i] + min_val) / 2)
                    elif crossed[i] > max_val:
                        corrected_vector = np.append(corrected_vector, (target[i] + max_val) / 2)
                    else:
                        corrected_vector = np.append(corrected_vector, crossed[i])

                surr += 1
                fit_new = fit_model(corrected_vector.reshape(1, -1))

                if fit_new < fitness_surr[idx]:                        
                    Sf[count] = F
                    Scr[count] = crossover_prob
                    d_f[count] = fitness_surr[idx] - fit_new
                    population_surr[idx] = corrected_vector
                    fitness_surr[idx] = fit_new
                    count += 1


                weights = np.zeros(pop_size)
                sum_d_f = np.sum(d_f[:count])
                if sum_d_f > 0:
                    for c in range(count):
                        weights[c] = d_f[c] / sum_d_f

                    mean_1 = 0
                    mean_2 = 0

                    for c in range(count):
                        mean_1 += weights[c] * Sf[c] * Sf[c]
                        mean_2 += weights[c] * Sf[c]

                    MF[iter % H] = mean_1 / mean_2
                    mean_1 = 0
                    mean_2 = 0

                    for c in range(count):
                        mean_1 += weights[c] * Scr[c] * Scr[c]
                        mean_2 += weights[c] * Scr[c]

                    MCr[iter % H] = mean_1 / mean_2


        for ind in population_surr:
            if ind not in xt and calc < num_calculate:
                ind = ind.reshape(1, n)
                xt = np.vstack((xt, ind))
                fitness = objective_func(ind)
                calc += pop_size
                yt = np.vstack((yt, fitness))
            sm = KRG(theta0=[1e-2], print_global = False)
            sm.set_training_values(xt, yt)
            sm.train()
    best_solution = population[np.argmin(fit_pop)]
    print(np.min(yt))

    return best_solution, np.min(fit_pop)

In [31]:
size = 20
n = 10
min_val = -100
max_val = 100
modes = 2
num_calculate = 400
num_exp = 51
num_surr = 1000

In [32]:
from cec2017.functions import f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f15, f16, f18, f19, f21, f22, f23, f24, f25, f26, f27, f28, f30

functions = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, 
             f13, f15, f16, f18, f19, f21, f22, f23, f24, f25, 
             f26, f27, f28, f30]
for mo in range(1, modes):
    for func in functions:
        fit_list = np.array([])
        for exp in range(num_exp):
            best_solution, fit = differential_evolution(mo, func, size, n, min_val, max_val, num_calculate, num_surr)
            fit_list = np.append(fit_list, fit)
        np.savetxt(f'result_{func.__name__}_mode_{mo}.txt', fit_list)
        print(f"{func.__name__} finished!")
    #print("Лучшее решение:", best_solution)
    #print("Значение функции:", fit)

213541082756.525
240686540989.74142


KeyboardInterrupt: 

In [34]:
from cec2017.functions import f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f15, f16, f18, f19, f21, f22, f23, f24, f25, f26, f27, f28, f30

print(f"Кол-во индивидов: {size}, Кол-во вычислений: {num_calculate}, n = {n}")
functions = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, 
             f13, f15, f16, f18, f19, f21, f22, f23, f24, f25, 
             f26, f27, f28, f30]

dict_mean = {}
dict_mean['ДЭ'] = []
dict_mean['ДЭ + SVR'] = []
for func in functions:
    data1 = np.loadtxt(f'result_{func.__name__}_mode_0.txt')
    data2 = np.loadtxt(f'result_{func.__name__}_mode_1.txt')
    mean_value1 = np.mean(data1)
    mean_value2 = np.mean(data2)
    dict_mean['ДЭ'].append(mean_value1)
    dict_mean['ДЭ + SVR'].append(mean_value2)

mean_df = pd.DataFrame(dict_mean, index=[f'{func.__name__}' for func in functions])

print(mean_df)

Кол-во индивидов: 20, Кол-во вычислений: 400, n = 10


FileNotFoundError: result_f2_mode_1.txt not found.